In [7]:
####### This code retrieves inventor location from the patent description  ####

####### Globals
setwd(dir = "../../")
inputpath<-"Marshall_2/ori_data/"
temp<-"Marshall_2/int_data/temp_clean_patent_all/"
outpath <- "Marshall_2/final_data/hist_US/"


Sys.setenv('R_MAX_VSIZE'=100000000000)


############ Packages ############ 
# Define a custom library directory in your home directory
user_lib <- "~/R/library"

# Create the directory if it doesn't exist
if (!dir.exists(user_lib)) {
  dir.create(user_lib, recursive = TRUE)
}

# Update library paths to include the new directory
.libPaths(user_lib)

# Verify the current library paths
print(.libPaths())

# Define a list of all the packages you want to load/install
packages <- c(
  "slam", "parallel", "jurithy", "RecordLinkage", "stringdist", 
  "qdapRegex", "devtools", "openxlsx", "hunspell", "textstem", 
  "sjmisc", "rvest", "stringi", "dplyr", "lubridate", 
  "readr", "tm", "base", "data.table", "stopwords", 
  "textstem", "ngram", "stringr", "networkD3", 
  "rcorpora", "pluralize", "Matrix", "tidyr"
)

# Iterate through each package and install them
for (pkg in packages) {
  tryCatch(
    {
      # Try loading the library
      library(pkg, character.only = TRUE)
      message(paste("Successfully loaded package:", pkg))
    },
    error = function(e) {
      # If there's an error loading the package, try to install it in the user-defined library
      message(paste("Package", pkg, "not found. Attempting to install..."))
      tryCatch(
        {
          install.packages(pkg, lib = user_lib)
          library(pkg, character.only = TRUE)
          message(paste("Successfully installed and loaded package:", pkg))
        },
        error = function(e) {
          # Handle installation failure
          message(paste("Failed to install package:", pkg, "Error:", e$message))
        }
      )
    }
  )
}


############   1. Retrieve patent description and author/assignee name(s) and perform first cleaning ############   

############  1.0. Selection based on localised inventors  ############ 
load(paste0(temp, "/locationData2.RData"))
data_des<-data.frame(subset(data, select=c("i", "description")))
names(data_des)[names(data_des) == "i"] <- "Patent_ID"
relevant_clean <- merge(relevant_clean,data_des,by="Patent_ID")
rm(list = setdiff(ls(), "relevant_clean"))
data<-relevant_clean
# inputpath<-"/Users/filippoboeri/Library/CloudStorage/OneDrive-LondonSchoolofEconomics/Marshall in Marshall times/Data/Input/Patent/"
# temp<-"Marshall_2/int_data/temp"
# outpath <- "/Users/filippoboeri/Library/CloudStorage/OneDrive-LondonSchoolofEconomics/Marshall in Marshall times/Data/Output/"
# setwd(dir = "/Users/filippoboeri/Library/CloudStorage/OneDrive-LondonSchoolofEconomics/Marshall in Marshall times/Data/Temp/")

[1] "/efs/users/xiet13/R/library"                                  
[2] "/zfs/apps/hand/miniconda/23.5.2/envs/r-devtools/lib/R/library"


Package slam not found. Attempting to install...

Warning message in install.packages(pkg, lib = user_lib):
“installation of package ‘slam’ had non-zero exit status”
Failed to install package: slam Error: there is no package called ‘slam’

Successfully loaded package: parallel

Package jurithy not found. Attempting to install...

Warning message:
“package ‘jurithy’ is not available for this version of R

A version of this package for your version of R might be available elsewhere,
see the ideas at
https://cran.r-project.org/doc/manuals/r-patched/R-admin.html#Installing-packages”
Failed to install package: jurithy Error: there is no package called ‘jurithy’

Package RecordLinkage not found. Attempting to install...

Warning message:
“dependencies ‘MASS’, ‘Matrix’ are not available”
also installing the dependencies ‘listenv’, ‘parallelly’, ‘codetools’, ‘future’, ‘globals’, ‘shape’, ‘future.apply’, ‘numDeriv’, ‘progressr’, ‘SQUAREM’, ‘diagram’, ‘KernSmooth’, ‘lava’, ‘bit64’, ‘blob’, ‘plog

ERROR: Error in readChar(con, 5L, useBytes = TRUE): cannot open the connection


In [3]:
############ 1.1 Data
Inventors = read.csv(paste0(inputpath, "Authors.csv"))
names(Inventors)[names(Inventors) == "ID_patent"] <- "i"
Inventors_wide = read.csv(paste0(inputpath, "Authors_wide.csv"))
names(Inventors_wide)[names(Inventors_wide) == "ID_patent"] <- "i"
Titles = read.csv(paste0(inputpath, "Titles.csv"))
names(Titles)[names(Titles) == "ID_patent"] <- "i"
Occupations = read.csv(paste0(inputpath, "Occupation_string3.csv"))

In [5]:
############ 1.2 Clean occupation titles
Occupations$term <- gsub("- ", "", Occupations$term)
Occupations$term <- gsub("[[:punct:]]", " ", Occupations$term)
Occupations$term =gsub("([0-9]+)", "", Occupations$term)
Occupations$term=toupper(Occupations$term)
Occupations$term=singularize(Occupations$term)
Occupations$term =gsub("BRIDGE", "", Occupations$term)
Occupations$term =gsub("ETC", "", Occupations$term)
Occupations$term=stripWhitespace(Occupations$term)
Occupations$term=trimws(Occupations$term)

Occupations2<-Occupations
Occupations2$term=pluralize(Occupations2$term)
Occupations<-rbind(Occupations,Occupations2)
Occupations2<-NULL

Occupations<-unique(Occupations)
Occupations<-data.frame(Occupations)
Occupations <- data.frame(Occupations[ which(Occupations$term!=''), ])


ERROR: Error in singularize(Occupations$term): could not find function "singularize"


In [ ]:
############ 1.3 Clean inventor names
Inventors$names <- gsub("- ", "", Inventors$names)
Inventors$names <- gsub("[^[:punct:],]", " ", Inventors$names)
Inventors$names =gsub("([0-9]+)", "", Inventors$names)
Inventors$names=toupper(Inventors$names)
Inventors$names=stripWhitespace(Inventors$names)
Inventors$names=trimws(Inventors$names)

Inventors_wide$names <- gsub("- ", "", Inventors_wide$names)
Inventors_wide$names <- gsub("[^,[^:punct:]]", " ", Inventors_wide$names)
Inventors_wide$names <- gsub(" ,", ",", Inventors_wide$names)
Inventors_wide$names =gsub("([0-9]+)", "", Inventors_wide$names)
Inventors_wide$names=toupper(Inventors_wide$names)
Inventors_wide$names=stripWhitespace(Inventors_wide$names)
Inventors_wide$names=trimws(Inventors_wide$names)

In [ ]:

############ 1.4 Clean patent titles
Titles$title <- gsub("<RTI> & </RTI>", "", Titles$title)
Titles$title <- gsub("</RTI>", "", Titles$title)
Titles$title <- gsub("<RTI>", "", Titles$title)
Titles$title <- gsub("~", "", Titles$title)
Titles$title <- gsub("~", "", Titles$title)
Titles$title <- gsub("- ", "", Titles$title)
Titles$title <- gsub("[[:punct:]]", " ", Titles$title)
Titles$title=toupper(Titles$title)
Titles$title=stripWhitespace(Titles$title)
Titles$title=trimws(Titles$title)
data$description =gsub("([0-9]+)", "", data$description)

In [ ]:




############ 1.5 Merge
names(data)[names(data) == "Patent_ID"] <- "i"

data <- merge(data,Inventors_wide,by="i")
data <- merge(data,Titles,by="i")
data <- data[ which(data$description!='NA'), ]

# W1: 233,158,727
wordcount(data$description, sep = " ", count_fun = sum)
word_count<-data.frame('1')
word_count$w1<-wordcount(data$description, sep = " ", count_fun = sum)


############ 2 Merge Inventors: from 156,913 to 113,512
Found_inventors =read.csv(paste0(outpath, "Localised_inventors_v3.csv"))
names(Found_inventors)[1] <- "i"
data <- merge(data,Found_inventors,by="i")

# W2: 161,648,646
wordcount(data$description, sep = " ", count_fun = sum)
word_count$w2<-wordcount(data$description, sep = " ", count_fun = sum)



############  3. First cleaning  ############ 
data$description <- gsub("<RTI> & </RTI>", "", data$description)
data$description <- gsub("</RTI>", "", data$description)
data$description <- gsub("<RTI>", "", data$description)
data$description <- gsub("~", "", data$description)
data$description <- gsub("~", "", data$description)
data$description <- gsub("- ", "", data$description)
data$description <- gsub("[[:punct:]]", " ", data$description)
data$description=toupper(data$description)
data$description=stripWhitespace(data$description)
data$description=trimws(data$description)
data=data[which(data$description!=""),]
data$description <- gsub(" IMG ", "", data$description)
data$description <- gsub(" CLASS ", "", data$description)
data$description <- gsub(" EMIREF ", "", data$description)
data$description <- gsub(" ID ", "", data$description)
data$description <- gsub(" CLSS ", "", data$description)
data$description <- gsub("((//d{4,}))", "", data$description) 
data$description <- gsub("PROVISIONAL ", " ", data$description)
data$description <- gsub("PROVISIONL ", " ", data$description)
data$description <- gsub("PROVIIONAL ", " ", data$description)
data$description <- gsub("PRVISINAL ", " ", data$description)
data$description <- gsub("COMPLETE", " ", data$description)
data$description <- gsub(" SPECIFICATION ", " ", data$description)
data$description <- gsub(" SPECIFCATION ", " ", data$description)
data$description <- gsub(" PECIFICATION ", " ", data$description)
data$description <- gsub(" SPECIFICTION ", " ", data$description)
data$description <- gsub(" SPECIFICATIN ", " ", data$description)
data$description <- gsub(" IMPROVEMENTS ", " ", data$description)
data$description <- gsub(" IMPROVED ", " ", data$description)
data$description <- gsub(" THE NATURE OF THIS INVENTION ", " ", data$description)
data$description <- gsub(" TO BE AS FOLLOWS ", " ", data$description)
data$description <- gsub(" RTI ", " ", data$description)
data$description <- gsub("  ", " ", data$description)
data$description <- gsub("[0-9]{4}(.*?)", "", data$description)
data$description <- gsub("000", "", data$description)
data$description =gsub("([0-9]+)", "", data$description)

# W3: 158,442,323
wordcount(data$description, sep = " ", count_fun = sum)
word_count$w3<-wordcount(data$description, sep = " ", count_fun = sum)


############  4. REMOVE INTRO, LOCATION NAMES, STOPWORDS   ############ 


#### remove intro
data$pos2 <- regexpr('DO HEREBY DECLARE', data$description) # Returns position of 1st match in a string
data$max<-str_length(data$description)
data$pos2[which(data$pos2==data$max)] <-1
data$pos2[which(data$pos2>500)] <-1
data$description<- substr(data$description, data$pos2, data$max )
data$description <- gsub("DO HEREBY DECLARE", " ", data$description)

#### remove town_names
locations1<-data.frame(data$county)
locations2<-data.frame(data$district)
locations3<-data.frame(data$subdistrict)
locations4<-data.frame(data$parish)
locations5<-data.frame(data$city)
locations6<-data.frame(data$town)

names(locations1)[1] <- "location"
names(locations2)[1] <- "location"
names(locations3)[1] <- "location"
names(locations4)[1] <- "location"
names(locations5)[1] <- "location"
names(locations6)[1] <- "location"
town<-rbind( locations1,locations2,locations3,locations4,locations5, locations6)

towns_unique<-data.frame(unique(town))
data$description <- removeWords(data$description, towns_unique$town)




#### Remove stopwords
data$description <- gsub("NATURE", " ", data$description)
data$description <- gsub("INVENTION", " ", data$description)

data$description <- gsub("PROVISIONAL SPECIFICATION", " ", data$description)
data$description <- gsub("COMPLETE SPECIFICATION", " ", data$description)
data$description <- gsub(" RTI ", " ", data$description)
data$description <- gsub("  ", " ", data$description)
data$description=removeWords(data$description, "")
data$description <- gsub("~", " ", data$description)


SW <- stopwords("en", source = "snowball")
SW=trimws(stripWhitespace(toupper(removePunctuation(SW))))  
SW
data$description=removeWords(data$description, SW)

SW <- stopwords("en", source = "smart")
SW=trimws(stripWhitespace(toupper(removePunctuation(SW))))  
data$description=removeWords(data$description, SW)
data$description=removeNumbers(data$description)

#### Remove duplicates within each cell
data$description<-vapply(lapply(strsplit(data$description, " "), unique), paste, character(1L), collapse = " ")

# W4: 24,713,595
wordcount(data$description, sep = " ", count_fun = sum)
word_count$w4<-wordcount(data$description, sep = " ", count_fun = sum)



############  5. REMOVE TERMS THAT APPER ONCE OR TWICE   ############ 

termFreq<-data.frame(termFreq(data$description, control = list()))
setDT(termFreq, keep.rownames = TRUE)[]
termFreq$rn <- gsub("[[:punct:]]", " ", termFreq$rn)
termFreq$rn=toupper(termFreq$rn)

termFreq$rn=removeWords(termFreq$rn, "~")
names(termFreq)[2] <- "freq"
termFreq <- termFreq[order(-freq),]
termFreq[1:100, ]

wordcount(data$description, sep = " ", count_fun = sum)
st<-data.frame(termFreq$rn[termFreq$freq<=2])
singleterms<-NULL
singleterms<-data.frame(st)
names(singleterms)[1] <- "terms"

group=1000
n <- length(singleterms$terms)
r <- rep(1:ceiling(n/group),each=group)[1:n]
d <- split(singleterms$terms,r)

for (i in 1:length(d)) {
  data$description <- removeWords(data$description, c(paste(d[[i]])))
}



#W5: 24,657,802
wordcount(data$description, sep = " ", count_fun = sum)
word_count$w5<-wordcount(data$description, sep = " ", count_fun = sum)

############  6. REMOVE N MOST FREQUENT NON-TECHNICAL WORDS (1)  ############ 

V1=c("RTI", "INVENTION", "END", "FIG", "FIGURE", "MEANS", "OPEN", "PROCESS", "PERMANENT", "TAKE", "MAKE", "THREAD", "STATE", "PROCEED", "THEREOF", "ORDINARY", "USUAL", "CREATE", "DO", "MODE", "TREAT", "ELIMINATE", "FREE", "PRODUCE", "PREFER", "BOX", "FEET", "TH", "INTRODUCE", "POSITION", "CLASS", "SIDE", "IMG", "EMIREF", "PROVIDED", "MANNER", "SHOWN", "PART", "SUITABLE", "FORM", "CONNECTED", "FORMED", "NATURE", "SECTION", "ENDS","SPECIFICATION", "LOWER", "DECLARE", "IMPROVEMENTS", "IMPROVED", "IMPROVE", "ARRANGED", "VIEW", "FIXED", "PORTION", "CONTACT", "SECURED", "LINE", "MOVEMENT", "DESIRED", "FRONT", "BACK", "BOTTOM", "DEVICE", "COMPLETE", "CONSTRUCTION", "DRAWING", "ORDER", "VERTICAL", "HORIZONTAL", "MOUNTED", "ARRANGEMENT", "ATTACHED", "TIME", "DIRECTION", "PURPOSE", "ACTION", "PLACE", "SIMILAR", "FIGS", "STATEMENT") 
data$description=removeWords(data$description, V1)

#W6 21,996,954
wordcount(data$description, sep = " ", count_fun = sum)
word_count$w6<-wordcount(data$description, sep = " ", count_fun = sum)




############ 7. Remove inventor names  ############ 
Inventors = read.csv(paste0(inputpath, "Authors.csv"))
names(Inventors)[names(Inventors) == "ID_patent"] <- "i"
Inventors <- merge(Inventors,data,by="i")

Inventors$names<-Inventors$names.x
Inventors$names <- gsub("- ", "", Inventors$names)
Inventors$names <- gsub("[[:punct:]]", " ", Inventors$names)
Inventors$names=toupper(Inventors$names)
Inventors$names=stripWhitespace(Inventors$names)
Inventors$names=trimws(Inventors$names)
Inventors_unique<-unique(Inventors$names)

group=1000
n <- length(Inventors_unique)
r <- rep(1:ceiling(n/group),each=group)[1:n]
d <- split(Inventors_unique,r)

for (i in 1:length(d)) {
  data$description <- removeWords(data$description, c(paste(d[[i]])))
}

# W7: 21,830,725
wordcount(data$description, sep = " ", count_fun = sum)
word_count$w7<-wordcount(data$description, sep = " ", count_fun = sum)



############ 8. LEMMATIZE THE TEXT   ############ 

data2<-data.frame(data)
data$description<-lemmatize_strings(data$description)
data$description<-toupper(data$description)

V1<-lemmatize_words(V1)
V1<-toupper(V1)
data$description=removeWords(data$description, V1)

data$description<-vapply(lapply(strsplit(data$description, " "), unique), paste, character(1L), collapse = " ")

#W8 18,327,148
wordcount(data$description, sep = " ", count_fun = sum)
word_count$w8<-wordcount(data$description, sep = " ", count_fun = sum)



############  9. REMOVE WORDS WITH LESS THAN 3 LETTERS   ############ 
data$description <- gsub(" [a-zA-Z] ", " ", data$description)
data$description <- gsub(" *\\b[[:alpha:]]{1,2}\\b * ", " ", data$description)

#W9 17,913,754
wordcount(data$description, sep = " ", count_fun = sum)
word_count$w9<-wordcount(data$description, sep = " ", count_fun = sum)


############  10. REMOVE N MOST FREQUENT NON-TECHNICAL WORDS (2)  ############ 

termFreq<-data.frame(termFreq(data$description, control = list()))
setDT(termFreq, keep.rownames = TRUE)[]
termFreq$rn <- gsub("[[:punct:]]", " ", termFreq$rn)
termFreq$rn=toupper(termFreq$rn)

termFreq$rn=removeWords(termFreq$rn, "~")
termFreq$rn <- gsub("~", "", termFreq$rn)
termFreq$rn <- gsub("~", "", termFreq$rn)
names(termFreq)[2] <- "freq"
termFreq <- termFreq[order(-freq),]
termFreq[1:100, ]

V1=c("PERFORM", "ASCERTAIN", "ACCOMPANY", "RELATE", "OBJECT", "CARRY", "PASS", "CONSIST", "COUNTY", "EFFECT", "REFERENCE", "UPPER", "REQUIRE", "PLAN", "WORK", "ILLUSTRATE", "TOP", "SMALL", "PIECE", "REFER", "PRESENT", "APPLY", "CONNECTION", "OPOSITE", "ACT", "PROJECT", "LARGE", "HAND", "READILY", "CAUSE", "SEPARATE") 
data$description=removeWords(data$description, V1)

V1=c("PREVENT", "ENGINEER", "LEAVE", "DISTANCE", "OBTAIN", "CENTRE", "THERETO", "SERVE", "LONGITUDINAL", "INCREASE", "REDUCE", "PARTLY", "COUJTY", "ALLOW", "SIMPLE", "REAIN", "CITY", "STREET", "AFORESAID", "HALF", "DIRECTLY", "DESCRIPTION", "MODIFY", "ADVANTAGE", "TRAVEL", "SUFFICIENTLY", "HERETOFORE", "EFFICIENT", "ENSURE", "REGULAR", "OVERCOME", "IMMEDIATELY", "PND", "DIRECTOR", "UNDERNEATH", "INTEND")
data$description=removeWords(data$description, V1)

V1=c("DRAWING", "PREFERABLY", "EMPLOY", "FIT", "SHAPE", "TURN", "CONSTRUCT", "MATERIAL", "SUPPORT", "UNDERSTAND", "ENGINEER", "MOVE", "SURFACE", "ADAPT", "REMOVE", "EXTEND", "SET", "LEVAVE", "OPPOSITE", "OPERATE", "OUTER", "RECEIVE", "CLOSE", "ENGAGE", "POINT", "OPERATION", "LENGTH", "CASE", "COVER", "PIN", "REPRESENT", "LETTER", "ENABLE", "SPACE", "REST", "BRING", "CONVENIENT", "HEREINAFET", "DRIVE", "RAISE", "DETAIL", "METHOD", "HOLD", "SUFFICIENT", "SIZE", "ENTER", "VARY", "CLAIM", "AIR", "FIND", "RUN", "DESCRIBE", "PERMIT", "ADJUST", "GREAT", "HOLE", "INSERT", "PROPER", "GIVE", "HANDLE", "MECHANISM", "HIGH", "RESULT", "COMMUNICATE", "INSTANCE", "FULLY", "LIE", "SLIGHTLY", "INSIDE")
data$description=removeWords(data$description, V1)

data$description=stripWhitespace(data$description)
data$description=trimws(data$description)

data$description<-stem_strings(data$description)
data$description<-vapply(lapply(strsplit(data$description, " "), unique), paste, character(1L), collapse = " ")

#W10 13,726,425
wordcount(data$description, sep = " ", count_fun = sum)
word_count$w10<-wordcount(data$description, sep = " ", count_fun = sum)



##########################  11. Keep only words found in the dictionary ########################################################
dictionary <- read.delim(paste0(inputpath,"words_alpha.txt"))

dictionary$X2 <- gsub("[[:punct:]]", " ", dictionary$a)
dictionary$X2=toupper(dictionary$X2)
dictionary$X2<-lemmatize_strings(dictionary$X2)

termFreq2<-data.frame(termFreq(data$description, control = list()))
setDT(termFreq2, keep.rownames = TRUE)[]
termFreq2$rn <- gsub("[[:punct:]]", " ", termFreq2$rn)
termFreq2$rn=toupper(termFreq2$rn)
termFreq2$rn <- gsub("~", "", termFreq2$rn)
termFreq2$rn <- gsub("~", "", termFreq2$rn)

names(termFreq2)[1] <- "X2"

dictionary$correct<-'YES'
dictionary_words <- merge(dictionary,termFreq2,by="X2", all.y=TRUE)
wordstoberemoved<-dictionary_words[which(is.na(dictionary_words$correct)),]

#### 457,000 words, only 37,000 found in dictionary

st<-data.frame(wordstoberemoved$X2)
singleterms<-NULL
singleterms<-data.frame(st)
names(singleterms)[1] <- "terms"

group=1000
n <- length(singleterms$terms)
r <- rep(1:ceiling(n/group),each=group)[1:n]
d <- split(singleterms$terms,r)

for (i in 1:length(d)) {
  data$description <- removeWords(data$description, c(paste(d[[i]])))
}

data$description=stripWhitespace(data$description)
data$description=trimws(data$description)

#W11
wordcount(data$description, sep = " ", count_fun = sum)
word_count$w11<-wordcount(data$description, sep = " ", count_fun = sum)
termFreq3<-data.frame(termFreq(data$description, control = list()))




############  12. CONVERT DATAFRAME TO CORPUS   ############ 
termFreq2<-data.frame(termFreq(data$description, control = list()))
write.csv(data, file=paste0(outpath,"Input_patent_matrix.csv"), row.names=FALSE)